In [ ]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when

# Configuração para baixar o driver do Postgres automaticamente
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("ETL_Enem_Raw_To_Silver") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.2.18") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

print("Spark Sessão Ativa!")

In [ ]:
# Caminho absoluto do arquivo (Baseado no seu ambiente)
caminho_csv = "/home/emivalto/workspace/BD2V/SDBD2---INEP/raw/MICRODADOS_ENEM_2021.csv"

print(f"Lendo arquivo de: {caminho_csv}")

try:
    df_raw = spark.read \
        .option("header", "true") \
        .option("delimiter", ";") \
        .option("encoding", "ISO-8859-1") \
        .option("inferSchema", "true") \
        .csv(f"file://{caminho_csv}")
    
    print(f"Sucesso! Colunas encontradas: {len(df_raw.columns)}")
    # Mostra as primeiras linhas para você conferir se leu certo
    df_raw.show(2)

except Exception as e:
    print("ERRO NA LEITURA:")
    print(e)

In [ ]:
print("Aplicando transformações...")

df_silver = df_raw.select(
    col("NU_INSCRICAO").alias("id_inscricao"),
    col("NU_ANO").alias("ano"),
    col("TP_SEXO").alias("sexo"),
    col("NO_MUNICIPIO_PROVA").alias("cidade_prova"),
    col("SG_UF_PROVA").alias("uf_prova"),
    
    # Converte notas para FLOAT e trata nulos
    when(col("NU_NOTA_CN") == "NaN", 0.0).otherwise(col("NU_NOTA_CN").cast("float")).alias("nota_ciencias_natureza"),
    when(col("NU_NOTA_CH") == "NaN", 0.0).otherwise(col("NU_NOTA_CH").cast("float")).alias("nota_humanas"),
    when(col("NU_NOTA_LC") == "NaN", 0.0).otherwise(col("NU_NOTA_LC").cast("float")).alias("nota_linguagens"),
    when(col("NU_NOTA_MT") == "NaN", 0.0).otherwise(col("NU_NOTA_MT").cast("float")).alias("nota_matematica"),
    when(col("NU_NOTA_REDACAO") == "NaN", 0.0).otherwise(col("NU_NOTA_REDACAO").cast("float")).alias("nota_redacao")
).fillna(0.0)

print("Dados transformados! Visualizando amostra:")
df_silver.show(5)

In [ ]:
# Configurações do Banco
db_url = "jdbc:postgresql://localhost:5432/dados_inep"
usuario = "admin"
senha = "l1l2r1r2"
tabela_destino = "public.silver_enem"

print(f"Gravando dados na tabela {tabela_destino}...")

try:
    df_silver.write \
        .format("jdbc") \
        .option("url", db_url) \
        .option("dbtable", tabela_destino) \
        .option("user", usuario) \
        .option("password", senha) \
        .option("driver", "org.postgresql.Driver") \
        .mode("overwrite") \
        .save()
    
    print("✅ GRAVAÇÃO CONCLUÍDA COM SUCESSO!")

except Exception as e:
    print("❌ ERRO AO GRAVAR NO BANCO:")
    print(e)